✅ 수능 국어 문제를 준비합니다. 다음 github의 data > 2023_11_KICE.json data를 colab으로 불러오시면 됩니다:

In [1]:
import json

with open('./2023_11_KICE.json', 'r') as file:
    ko = json.load(file)

ko

In [2]:
pip -q install openai

Note: you may need to restart the kernel to use updated packages.


✅ 하나의 문제에 대해서 GPT-4의 예측 결과를 내놓는 함수를 def prediction(problem)이라는 signature로 만드셔야 합니다. problem은 json 형태의 문제입니다. 내부는 logit 계산을 통해 구현하거나 순수하게 text 생성으로 해결하셔도 좋습니다. 단, 2023년도 수능 국어의 정답을 활용하시면 안됩니다.

In [3]:
from openai import OpenAI
client = OpenAI()

def prediction(problem):
    res = []

    system_content = """You are a korean language teacher. Please choice the answer from the paragraph and questions. 
        ANSWER WITH ONLY ONE NUMBER DIGIT THAT YOU ASSUMED ANSWER, THE CHOICES INDEX IS START FROM 1, SO YOU CHOOSE SECOND, 
        THE OUTPUT IS NOT 1. NOT USING PROGRAMMING ARRAY INDEX START WITH 0"""


    for question in problem["problems"]:
        user_content = f"Paragraph: {problem["paragraph"]}, Question: {question}"

        if 'question_plus' in problem:
            user_content += f", Question Plus: {question['question_plus']}"
        
        user_content += f", Choices: {question["choices"]}"
        
        messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content": user_content}
        ]

        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )

        res.append(completion.choices[0].message.content)

    return res

In [4]:
prds = [prediction(k) for k in ko]

In [6]:
ans = [[pb["answer"], pb["score"]] for item in ko for pb in item["problems"]]

In [7]:
prd = [int(item) for sublist in prds for item in sublist]

✅ 채점 결과 50점을 넘기면 통과입니다.

In [8]:
sum([a[1] for a, p in zip(ans, prd) if a[0] == p])

96

96점이므로 1등급.

데이터 셋 레포지토리에서 gpt-4o 벤치마크한 69점 4등급에 비해 훨씬 높은 점수 받음,  o1-preview의 88점 보다 원점수가 높음.